# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [99]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [100]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [101]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [102]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [103]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [104]:
df_trade_summ

,asset,feesPaid
0,AAVE,5103.803754
1,APE,30942.762385
2,ATOM,28587.359435
3,AUD,91.576363
4,AVAX,43355.443862
5,AXS,2684.826915
6,BNB,54748.443868
7,DOGE,11069.138967
8,DYDX,36646.745044
9,EUR,2236.859000


## Get the market data

In [105]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [106]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,8.500486e+06
1,sBTC,7.127760e+06
2,LINK,1.916660e+05
3,SOL,4.940315e+05
4,AVAX,2.657966e+05
5,AAVE,2.651844e+03
6,UNI,1.024739e+05
7,MATIC,4.080087e+05
8,APE,6.626546e+04
9,DYDX,4.052227e+04


In [107]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,8.500486e+06,9.262403e+06,966014.408859,-1.727931e+06
1,sBTC,7.127760e+06,7.506937e+06,849472.904913,-1.228649e+06
2,LINK,1.916660e+05,2.794874e+05,96224.746157,-1.840462e+05
3,SOL,4.940315e+05,5.680611e+05,117727.692236,-1.917572e+05
4,AVAX,2.657966e+05,1.963304e+05,43355.443862,2.611082e+04
5,AAVE,2.651844e+03,7.721074e+02,5103.803754,-3.224067e+03
6,UNI,1.024739e+05,1.152431e+05,10756.227668,-2.352542e+04
7,MATIC,4.080087e+05,5.144117e+05,90782.041864,-1.971850e+05
8,APE,6.626546e+04,3.726012e+04,30942.762385,-1.937424e+03
9,DYDX,4.052227e+04,6.437749e+04,36646.745044,-6.050196e+04


In [108]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [109]:
df_pnl['netPnl'].sum()

-4192588.865175258

In [110]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1727931.06
1,sBTC,-1228649.42
11,OP,-314186.82
7,MATIC,-197185.02
3,SOL,-191757.22
2,LINK,-184046.15
19,FTM,-134761.83
10,BNB,-67522.54
9,DYDX,-60501.96
16,ATOM,-48370.44
